# Third candidate score

### Let's iterate the analisys with other candidates:

### First, we must build a database with venues 800m around the candidate:

In [1]:
import pandas as pd
import json
from pymongo import MongoClient
from pymongo import GEOSPHERE

In [2]:
from src import api_functions as af
from src import cleaning_functions as cf

### let's bring our third candidate...

In [3]:
df = pd.read_csv("DATA/candidates.csv")
place=df.iloc[2]                                      # change this for another place
place

Unnamed: 0                2
city          San Francisco
state                    CA
latitude          37.798853
longitude       -122.398599
Name: 2, dtype: object

### ... and build a database of venues around it:

In [4]:
location = f"{place.latitude},{place.longitude}"
venues=[]
radio=800

In [5]:
requi=["vegan","kindergarten","Starbucks","bar"]

In [6]:
for req in requi:
    resp=af.google(req,radio,location)
    clean = cf.limpiame(resp,req)
    for element in clean:
        venues.append(element)

In [8]:
name="place2.json"                                      # change this for another place

In [9]:
with open(f"DATA/{name}", 'w') as f:
    json.dump(venues, f)

### let's move to mongoDB...

In [10]:
client = MongoClient("localhost:27017")
db = client.get_database("FindLoc")

In [11]:
places = db.get_collection("place2")                     # change this for another place

In [12]:
db.place2.create_index([("location", GEOSPHERE)])        # change this for another place

'location_2dsphere'

### ... and make some queries:

In [13]:
locationfloat=[float(e) for e in location.split(",")]
locationreverse=[locationfloat[1],locationfloat[0]]
coord={"type":"Point", "coordinates": locationreverse}
proj = {"_id":0,"tipo":1,"nombre":1}
meters=500

In [14]:
requi

['vegan', 'kindergarten', 'Starbucks', 'bar']

In [15]:
weights=[10,8,5,2]

In [16]:
score=[]
for req in requi:
    cond = {"tipo":req}
    query = {"location": {"$near": {"$geometry": coord, "$maxDistance": meters}}}
    lis = list(places.find({"$and":[cond,query]},proj))
    score.append(len(lis))

In [17]:
score

[0, 1, 3, 3]

### For a total score of:

In [18]:
if score[2]==0:
    print ("At least 1 Starbucks is mandatory!")
    total_score=0
else:
    total_score = sum([a*b for a,b in zip(score,weights)])

In [19]:
total_score

29

## That's it!! Amazing place! ;)

## Bonus: mapping the venues

In [20]:
import folium

In [21]:
mymap = folium.Map(location=locationfloat, zoom_start=15, tiles="cartodbpositron")

In [22]:
icono = folium.Icon(color="red",icon_color="white",)
candi = folium.Marker(location=[place.latitude,place.longitude], tooltip="candidate",icon=icono)
candi.add_to(mymap);

In [23]:
rad = folium.Circle(location=[place.latitude,place.longitude],radius=500,color="green")
rad.add_to(mymap);

In [24]:
query = {"location": {"$near": {"$geometry": coord, "$maxDistance": meters}}}
proj = {"_id":0,"tipo":1,"nombre":1,"location.coordinates":1}
sitios = list(places.find(query,proj))

In [25]:
for element in sitios:
    mark = folium.Marker(location=[element["location"]["coordinates"][-1],element["location"]["coordinates"][0]], tooltip=element["nombre"])
    mark.add_to(mymap)

In [26]:
mymap